# Basic workflow with ManGO

## Authentication with iCommands

<div class="alert alert-block alert-info">
<h3>Every seven days</h3>
    
1. Go to https://mango.kuleuven.be/
2. In the tab of your zone, click on "How to connect"
3. Copy the snippet provided under "iCommands Client on Linux".

<font size=3>Then **paste the snippet** in the cell below, right under `%%bash`, like the (anonimized) example below.</font>

(Replacing the `USERNAME` with your username and the `TOKEN` with the password provided by "How to connect" should also work.)

**You don't need to do this every time: The authentication lasts 7 days.**
</div>

In [16]:
%%bash
mkdir -p ~/.irods
cat > ~/.irods/irods_environment.json <<'EOF'
{
    "irods_host": "ghum.irods.icts.kuleuven.be",
    "irods_port": 1247,
    "irods_zone_name": "ghum",
    "irods_authentication_scheme": "pam_password",
    "irods_encryption_algorithm": "AES-256-CBC",
    "irods_encryption_salt_size": 8,
    "irods_encryption_key_size": 32,
    "irods_encryption_num_hash_rounds": 8,
    "irods_user_name": "USERNAME",
    "irods_ssl_ca_certificate_file": "",
    "irods_ssl_verify_server": "cert",
    "irods_client_server_negotiation": "request_server_negotiation",
    "irods_client_server_policy": "CS_NEG_REQUIRE",
    "irods_default_resource": "default",
    "irods_cwd": "/ghum/home"
}
EOF
iinit -h | grep Version | grep -v -q 4.2. || sed -i 's/"irods_authentication_scheme": "pam_password"/"irods_authentication_scheme": "PAM"/' ~/.irods/irods_environment.json
echo 'TOKEN' | iinit --ttl 168 >/dev/null && echo You are now authenticated to irods. Your session is valid for 168 hours.
ils

You are now authenticated to irods. Your session is valid for 168 hours.
/ghum/home:
  C- /ghum/home/ghum_pilot034
  C- /ghum/home/ghum_pilot034_readonly
  C- /ghum/home/public
  C- /ghum/home/rdm_mango_training
  C- /ghum/home/u0022329
  C- /ghum/home/u0118974


## Setting up

Set up ManGO and load any other libraries you need.

In [17]:
import os
import ssl
from irods.session import iRODSSession
try:
    env_file = os.environ['IRODS_ENVIRONMENT_FILE']
except KeyError:
    env_file = os.path.expanduser('~/.irods/irods_environment.json')

ssl_context = ssl.create_default_context(
        purpose=ssl.Purpose.SERVER_AUTH,
        cafile=None, capath=None, cadata=None
        )
ssl_settings = {'ssl_context': ssl_context}

Since we are working interactively we will create an `irods.session.iRODSSession` object and then close it at the end of the notebook with `session.cleanup()`. If you were working on a script, you could run all your code inside a `with` statement.

In [18]:
session = iRODSSession(irods_env_file=env_file, **ssl_settings)

The final step to set up your environment is to define your working directory in a variable. For this notebook, it's "/ghum/home/ghum_pilot034/", but for your own project it might be **different**.

In [19]:
home_dir = "/ghum/home/ghum_pilot034/"

In [20]:
dataset_dir = home_dir + "DATASET_6" # it might be a different name
output_dir = home_dir + "TEAM_1" # it might be a different name

## WRITE YOUR CODE BELOW THIS

In [21]:
coll = session.collections.get(dataset_dir)
coll

<iRODSCollection 41023 b'DATASET_6'>

In [13]:
obj1 = session.data_objects.get(coll.subcollections[0].subcollections[0].data_objects[0].path)
obj1

<iRODSDataObject 41077 FL10589076_DIGI_0030_0110.pdf>

In [18]:
obj2 = session.data_objects.get(coll.subcollections[2].subcollections[0].data_objects[0].path)
obj2

<iRODSDataObject 1265923 FL10589090_DIGI_0030_0111.pdf>

In [20]:
obj1.metadata.items()

[<iRODSMeta 41098 GLOBUS::MD5 fb05a2acc18ef056983153d0c89462fc 1674827083>]

In [21]:
obj2.metadata.items()

[]

In [23]:
session.data_objects.get(obj1.path, 'obj1.pdf')

<iRODSDataObject 41077 FL10589076_DIGI_0030_0110.pdf>

In [24]:
session.data_objects.get(obj2.path, 'obj2.pdf')

<iRODSDataObject 1265923 FL10589090_DIGI_0030_0111.pdf>

## CLEAN UP AT THE END

In [5]:
# leave this cell at the end and running every time you are done
session.cleanup()